In [34]:
import os
import requests
import tiktoken
import numpy as np
import torch
import torch.nn as nn
from torch.nn import functional as F
import regex as re

In [3]:
input_file_path = os.path.join(os.getcwd() , 'input.txt')
# print(input_file_path)
# if not os.path.exists(input_file_path):
#     data_url = 'https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt'
#     with open(input_file_path, 'w', encoding='utf-8') as f:
#         f.write(requests.get(data_url).text)

input_file_path

'd:\\Project src\\zero-to-hero\\nanogpt\\input.txt'

In [4]:
with open('input.txt' , 'r' , encoding="utf-8") as f:
    text = f.read()

len(text)

1115394

In [4]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [5]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [6]:
stoi = {s : i for i , s in enumerate(chars)}
itos = {i : s for s , i in stoi.items()}

#take a string , output a list of integers
encode = lambda s: [stoi[c] for c in s]
#take a list of integers , output a string
decode = lambda l: ''.join([itos[i] for i in l])
# print(stoi)
# print(itos)
l = encode("hello world!")
print([itos[i] for i in l])

print(encode("hello world!"))
print(decode(encode("hello world!")))

['h', 'e', 'l', 'l', 'o', ' ', 'w', 'o', 'r', 'l', 'd', '!']
[46, 43, 50, 50, 53, 1, 61, 53, 56, 50, 42, 2]
hello world!


In [7]:
import tiktoken

#openai提供的编码器
enc_gpt2 = tiktoken.get_encoding("gpt2")
print(enc_gpt2.n_vocab)
encoding_words = enc_gpt2.encode("hello world!")
print(encoding_words)
print(enc_gpt2.decode(encoding_words))

50257
[31373, 995, 0]
hello world!


In [8]:
data = torch.tensor(encode(text) , dtype=torch.long)
print(data.shape , data.dtype)
print(data[:1000])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [9]:
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]
print(len(train_data) , len(val_data))

1003854 111540


In [10]:
block_size = 8

x = train_data[:block_size]
y = train_data[1 : block_size + 1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f'When input is {context} , the target is {target}')

When input is tensor([18]) , the target is 47
When input is tensor([18, 47]) , the target is 56
When input is tensor([18, 47, 56]) , the target is 57
When input is tensor([18, 47, 56, 57]) , the target is 58
When input is tensor([18, 47, 56, 57, 58]) , the target is 1
When input is tensor([18, 47, 56, 57, 58,  1]) , the target is 15
When input is tensor([18, 47, 56, 57, 58,  1, 15]) , the target is 47
When input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) , the target is 58


In [11]:
torch.manual_seed(1337)

batch_size = 4
block_size = 8


def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(0 , len(data) - block_size , (batch_size,))
    #torch.stack: cat the input sequence
    x = torch.stack([data[i: i + block_size] for i in ix])
    y = torch.stack([data[i + 1 : i + 1 + block_size] for i in ix])
    return x , y

xb , yb = get_batch('train')
print(f"input: " )
print(xb.shape)
print(xb)
print("target: ")
print(yb.shape)
print(yb)

print("------------------------------------------------")

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b , :t+1]   
        target = yb[b , t]
        print(f'When input is {context} , the target is {target}')


input: 
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
target: 
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
------------------------------------------------
When input is tensor([24]) , the target is 43
When input is tensor([24, 43]) , the target is 58
When input is tensor([24, 43, 58]) , the target is 5
When input is tensor([24, 43, 58,  5]) , the target is 57
When input is tensor([24, 43, 58,  5, 57]) , the target is 1
When input is tensor([24, 43, 58,  5, 57,  1]) , the target is 46
When input is tensor([24, 43, 58,  5, 57,  1, 46]) , the target is 43
When input is tensor([24, 43, 58,  5, 57,  1, 46, 43]) , the target is 39
When input is tensor([44]) , the target is 53
When input is tensor([44, 53]) , t

In [12]:
class BigramLanguageModel(nn.Module):
    def __init__(self , vocab_size):
        super().__init__()
        #Embedding将词汇表中的词汇映射到多维向量中，矩阵行数表示词汇表大小，矩阵列数表示每个token的向量维度
        #用embedding层作为二元语言模型的表，生成的词只依赖于前一个词
        self.token_embedding_table = nn.Embedding(vocab_size , vocab_size)

    def forward(self , idx , targets=None):
        logits = self.token_embedding_table(idx)
        
        if targets is None:
            loss = None
        else:
            B,T,C = logits.shape
            logits = logits.view(B*T , C)
            targets = targets.view(B*T)
            #cross_entropy要求logtis为(sample_num , classes_num) targets(class of every sample,)
            # print(logits.shape , targets.shape) logtits.shape = 
            # (batch_size * block_size , vocab_size)

            loss = F.cross_entropy(logits , targets)

        return logits , loss
    
    def generate(self , idx , max_new_token):
        for _ in range(max_new_token):
            logits , loss = self(idx)
            logits = logits[: , -1 , :]          #提取时间维度上的最后一个位置
            probs = F.softmax(logits , dim=-1)
            idx_next = torch.multinomial(probs , num_samples=1)
            idx = torch.cat((idx , idx_next) , dim=1)
        return idx

m = BigramLanguageModel(vocab_size)
logits , loss = m(xb , yb)       #输入4*8 输出 4 * 8 * 56
print(logits.shape)
print(loss)

torch.Size([32, 65])
tensor(5.0364, grad_fn=<NllLossBackward0>)


In [13]:
idx = torch.zeros((1 , 1) , dtype=torch.long)
print(m.generate(idx , max_new_token=100).shape)  
print(decode(m.generate(idx , max_new_token=100)[0].tolist()))

torch.Size([1, 101])

k'.?nA!LgjgL&jcM-sZW!HFsWt'rv.x;z&hPyd'XHOxnAcj.erRjcLxxVFSrOpW.oeGJ Ai$yi&!mRfeopx,f.e..iBgxVrl-VTh


In [14]:
optimizer = torch.optim.AdamW(m.parameters() , lr=1e-3)

batch_size = 32
for step in range(10000):
    xb , yb = get_batch('train')

    logits , loss = m(xb , yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

2.574044942855835


In [15]:
print(decode(m.generate(idx , max_new_token=500)[0].tolist()))




ARERI ISOLBUS:
QUK: ast, plt t wadyotl
I bel qunganonoth he m he de avellis k'l, tond soran:

WI he toust are bott oe n t s d je hid t his IAces I my ig t
Ril'swoll e pupat inould candenothiqu heamer te
Wht s

MI wect!-lltherotheve t fe;
WAnd py;

PO t s ld tathat, ir V
IO thesecin teot tit ado ilorer.
Ply, d'stacoes, ld omat mealellly yererer EMEvesas ie IZEd pave mautoofareanerllleyomerer but?
The t,
Ith'dwitile w? beren to'd ff a atrts brey s

ESesenther:
Ithon f at par,
NRmamy an flictong 


In [16]:
#我们希望第i个位置生成的内容，只能看见前i-1个位置的内容而不是全部内容
torch.manual_seed(1337)
B,T,C = 4 , 8 , 2
x = torch.randn(B, T , C)
print(x.shape)

#实现1：
x_bow = torch.zeros((B , T , C))
for b in range(B):
    for t in range(T):
        xprev = x[b , :t+1]
        x_bow[b , t] = torch.mean(xprev , 0)

print(x.shape , x[0])
print(x_bow.shape , x_bow[0])

torch.Size([4, 8, 2])
torch.Size([4, 8, 2]) tensor([[ 0.1808, -0.0700],
        [-0.3596, -0.9152],
        [ 0.6258,  0.0255],
        [ 0.9545,  0.0643],
        [ 0.3612,  1.1679],
        [-1.3499, -0.5102],
        [ 0.2360, -0.2398],
        [-0.9211,  1.5433]])
torch.Size([4, 8, 2]) tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])


In [17]:
#实现2
torch.manual_seed(42)
# a = torch.ones(3 , 3)
a = torch.tril(torch.ones(3,3))
a = a / torch.sum(a , 1 ,keepdim=True)
print(a)
b = torch.randint(0 , 10 , (3 , 2)).float()
c = a @ b
print(a)
print(b)
print(c)
print(torch.tril(torch.ones(3 , 3))) #生成下三角

wei = torch.tril(torch.ones(T , T))
wei = wei / wei.sum(1 , keepdim=True)
x_bow2 = wei @ x      #(T * T) @ (B , T , C)  ---> (B , T , T) @ (B , T , C)
torch.allclose(x_bow , x_bow2)

tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])
tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])


True

In [18]:
# use Softmax
wei = torch.zeros((T , T))
tril = torch.tril(torch.ones(T , T))
wei = wei.masked_fill(tril == 0 , float('-inf'))    #填充
print(wei)
wei = F.softmax(wei , dim=-1)
print(wei)
xbow3 = wei @ x
torch.allclose(x_bow , xbow3)

tensor([[0., -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., 0., -inf, -inf, -inf],
        [0., 0., 0., 0., 0., 0., -inf, -inf],
        [0., 0., 0., 0., 0., 0., 0., -inf],
        [0., 0., 0., 0., 0., 0., 0., 0.]])
tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])


True

In [54]:
## self attention
torch.manual_seed(1337)
B , T , C = 4 , 8 , 32
x = torch.randn(B , T , C)

#single head self-attention
head_size = 16
key = nn.Linear(C , head_size , bias=False)
query = nn.Linear(C , head_size , bias=False)
value = nn.Linear(C , head_size , bias=False)
k = key(x)             # B * T * head_size
q = query(x)           # B * T * head_size

wei = q @ k.transpose(-2 , -1)   #B * T * head_size @ B * head_size * T ---> B * T * T


tril = torch.tril(torch.ones(T , T))
# wei = torch.zeros((T , T))
#对于语言模型，需要采用掩码attention，模型只能从过去的序列中获取信息
wei = wei.masked_fill(tril == 0 , float('-inf'))
wei = F.softmax(wei , dim=-1)
print(wei)
# out = wei @ x
v = value(x)
out = wei @ v

out.shape

tensor([[[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
         [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
         [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
         [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],

        [[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.1687, 0.8313, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.2477, 0.0514, 0.7008, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.4410, 0.0957, 0.3747, 0.0887, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0069, 0.0456, 0.0300, 0.7748, 0.1427, 0.0000, 0.0000, 0.0000],
         [0.0660, 0.089

torch.Size([4, 8, 16])

In [22]:
tril = torch.tril(torch.ones(8 , 8))
wei = torch.ones((4 , 8 , 8))
print(wei.masked_fill(tril[:8 , : 8] == 0 , float('-inf')))

tensor([[[1., -inf, -inf, -inf, -inf, -inf, -inf, -inf],
         [1., 1., -inf, -inf, -inf, -inf, -inf, -inf],
         [1., 1., 1., -inf, -inf, -inf, -inf, -inf],
         [1., 1., 1., 1., -inf, -inf, -inf, -inf],
         [1., 1., 1., 1., 1., -inf, -inf, -inf],
         [1., 1., 1., 1., 1., 1., -inf, -inf],
         [1., 1., 1., 1., 1., 1., 1., -inf],
         [1., 1., 1., 1., 1., 1., 1., 1.]],

        [[1., -inf, -inf, -inf, -inf, -inf, -inf, -inf],
         [1., 1., -inf, -inf, -inf, -inf, -inf, -inf],
         [1., 1., 1., -inf, -inf, -inf, -inf, -inf],
         [1., 1., 1., 1., -inf, -inf, -inf, -inf],
         [1., 1., 1., 1., 1., -inf, -inf, -inf],
         [1., 1., 1., 1., 1., 1., -inf, -inf],
         [1., 1., 1., 1., 1., 1., 1., -inf],
         [1., 1., 1., 1., 1., 1., 1., 1.]],

        [[1., -inf, -inf, -inf, -inf, -inf, -inf, -inf],
         [1., 1., -inf, -inf, -inf, -inf, -inf, -inf],
         [1., 1., 1., -inf, -inf, -inf, -inf, -inf],
         [1., 1., 1., 1., -inf,

In [23]:
#Layer Normalization
class LayerNorm1d:
    def __init__(self , dim , eps=1e-5 , momentum = 0.1):
        self.eps = eps
        self.gamma = torch.ones(dim)
        self.beta = torch.zeros(dim)

    def __call__(self , x):
        xmean = x.mean(1 , keepdim = True)
        xvar = x.var(1 , keepdim=True)
        xhat = (x - xmean) / torch.sqrt(xvar + self.eps)
        self.out = self.gamma * xhat + self.beta

        return self.out 

    def parameters(self):
        return [self.gamma , self.beta]
    
torch.manual_seed(1337)
module = LayerNorm1d(100)
x = torch.randn(32 , 100)
x = module(x)
x.shape

torch.Size([32, 100])

In [24]:
x[: , 0].mean() , x[: , 0].std()        #batchNorm 则为0，1

(tensor(0.1469), tensor(0.8803))

In [25]:
x[0 , :].mean() , x[0 , :].std()           #layerNorm为0 ，1

(tensor(-9.5367e-09), tensor(1.0000))

## Tokenizer

In [7]:
# unicode编码

[ord(x) for x in "我爱你中国,I love China!!!"]

[25105,
 29233,
 20320,
 20013,
 22269,
 44,
 73,
 32,
 108,
 111,
 118,
 101,
 32,
 67,
 104,
 105,
 110,
 97,
 33,
 33,
 33]

In [9]:
list("我爱你中国,I love China!!!".encode("utf-8"))

[230,
 136,
 145,
 231,
 136,
 177,
 228,
 189,
 160,
 228,
 184,
 173,
 229,
 155,
 189,
 44,
 73,
 32,
 108,
 111,
 118,
 101,
 32,
 67,
 104,
 105,
 110,
 97,
 33,
 33,
 33]

In [4]:
text = "Ｕｎｉｃｏｄｅ! 🅤🅝🅘🅒🅞🅓🅔‽ 🇺‌🇳‌🇮‌🇨‌🇴‌🇩‌🇪! 😄 "
tokens = text.encode("utf-8")
tokens = list(map(int , tokens))
print("-----------------")
print(text)
print("length: " , len(text))
print("-----------------")
print(tokens)
print("length: " , len(tokens))


-----------------
Ｕｎｉｃｏｄｅ! 🅤🅝🅘🅒🅞🅓🅔‽ 🇺‌🇳‌🇮‌🇨‌🇴‌🇩‌🇪! 😄 
length:  35
-----------------
[239, 188, 181, 239, 189, 142, 239, 189, 137, 239, 189, 131, 239, 189, 143, 239, 189, 132, 239, 189, 133, 33, 32, 240, 159, 133, 164, 240, 159, 133, 157, 240, 159, 133, 152, 240, 159, 133, 146, 240, 159, 133, 158, 240, 159, 133, 147, 240, 159, 133, 148, 226, 128, 189, 32, 240, 159, 135, 186, 226, 128, 140, 240, 159, 135, 179, 226, 128, 140, 240, 159, 135, 174, 226, 128, 140, 240, 159, 135, 168, 226, 128, 140, 240, 159, 135, 180, 226, 128, 140, 240, 159, 135, 169, 226, 128, 140, 240, 159, 135, 170, 33, 32, 240, 159, 152, 132, 32]
length:  108


In [13]:
def get_stats(ids):
    counts = {}
    for pair in zip(ids , ids[1:]):
        counts[pair] = counts.get(pair , 0) + 1
    return counts 

stats = get_stats(tokens)
# print(sorted(((v , k) for k , v in stats.items()) , reverse=True))

[(15, (240, 159)), (7, (226, 128)), (7, (159, 135)), (7, (159, 133)), (6, (239, 189)), (6, (140, 240)), (6, (128, 140)), (3, (32, 240)), (2, (33, 32)), (1, (239, 188)), (1, (189, 143)), (1, (189, 142)), (1, (189, 137)), (1, (189, 133)), (1, (189, 132)), (1, (189, 131)), (1, (189, 32)), (1, (188, 181)), (1, (186, 226)), (1, (181, 239)), (1, (180, 226)), (1, (179, 226)), (1, (174, 226)), (1, (170, 33)), (1, (169, 226)), (1, (168, 226)), (1, (164, 240)), (1, (159, 152)), (1, (158, 240)), (1, (157, 240)), (1, (152, 240)), (1, (152, 132)), (1, (148, 226)), (1, (147, 240)), (1, (146, 240)), (1, (143, 239)), (1, (142, 239)), (1, (137, 239)), (1, (135, 186)), (1, (135, 180)), (1, (135, 179)), (1, (135, 174)), (1, (135, 170)), (1, (135, 169)), (1, (135, 168)), (1, (133, 164)), (1, (133, 158)), (1, (133, 157)), (1, (133, 152)), (1, (133, 148)), (1, (133, 147)), (1, (133, 146)), (1, (133, 33)), (1, (132, 239)), (1, (132, 32)), (1, (131, 239)), (1, (128, 189))]


In [22]:
top_pair = max(stats , key=stats.get)
top_pair

#将ids替换为idx
def merge(ids , pair , idx):
    newids = []
    i = 0
    while i < len(ids):
        if i < len(ids) - 1 and ids[i] == pair[0] and ids[i + 1] == pair[1]:
            newids.append(idx)
            i += 2
        else:
            newids.append(ids[i])
            i += 1
    return newids

# print(merge([5 , 6 , 6 , 7 , 9 , 1] , (6 , 7) , 99))
tokens2 = merge(tokens , top_pair , 256)
print(tokens2)
print(f'length: {len(tokens2)}')

[239, 188, 181, 239, 189, 142, 239, 189, 137, 239, 189, 131, 239, 189, 143, 239, 189, 132, 239, 189, 133, 33, 32, 256, 133, 164, 256, 133, 157, 256, 133, 152, 256, 133, 146, 256, 133, 158, 256, 133, 147, 256, 133, 148, 226, 128, 189, 32, 256, 135, 186, 226, 128, 140, 256, 135, 179, 226, 128, 140, 256, 135, 174, 226, 128, 140, 256, 135, 168, 226, 128, 140, 256, 135, 180, 226, 128, 140, 256, 135, 169, 226, 128, 140, 256, 135, 170, 33, 32, 256, 152, 132, 32]
length: 93


In [26]:
vocab_size = 276
num_merges = vocab_size - 255
ids = list(tokens)

merges = {}
for i in range(num_merges):
    stats = get_stats(ids)
    pair = max(stats , key = stats.get)
    idx = 256 + i
    ids = merge(ids , pair , idx)
    print(f'merging {pair} into a new token {idx}')
    merges[pair] = idx

print("tokens length: " , len(tokens))
print("ids length: " , len(ids))
print(f"compression ratio: {len(tokens) / len(ids):.2f}X")

merging (240, 159) into a new token 256
merging (256, 133) into a new token 257
merging (226, 128) into a new token 258
merging (256, 135) into a new token 259
merging (239, 189) into a new token 260
merging (258, 140) into a new token 261
merging (261, 259) into a new token 262
merging (33, 32) into a new token 263
merging (239, 188) into a new token 264
merging (264, 181) into a new token 265
merging (265, 260) into a new token 266
merging (266, 142) into a new token 267
merging (267, 260) into a new token 268
merging (268, 137) into a new token 269
merging (269, 260) into a new token 270
merging (270, 131) into a new token 271
merging (271, 260) into a new token 272
merging (272, 143) into a new token 273
merging (273, 260) into a new token 274
merging (274, 132) into a new token 275
merging (275, 260) into a new token 276
[276, 133, 263, 257, 164, 257, 157, 257, 152, 257, 146, 257, 158, 257, 147, 257, 148, 258, 189, 32, 259, 186, 262, 179, 262, 174, 262, 168, 262, 180, 262, 169, 26

In [27]:
vocab = {idx:bytes([idx]) for idx in range(256)}
for (p0 , p1) , idx in merges.items():
    vocab[idx] = vocab[p0] + vocab[p1]

def decode(ids):
    tokens = b"".join(vocab[idx] for idx in ids)
    text = tokens.decode("utf-8" , errors="replace")      #默认为'strict'，此时在某些不符合格式要求的会报错
    return text

print(decode([128]))

�


In [28]:
merges

{(240, 159): 256,
 (256, 133): 257,
 (226, 128): 258,
 (256, 135): 259,
 (239, 189): 260,
 (258, 140): 261,
 (261, 259): 262,
 (33, 32): 263,
 (239, 188): 264,
 (264, 181): 265,
 (265, 260): 266,
 (266, 142): 267,
 (267, 260): 268,
 (268, 137): 269,
 (269, 260): 270,
 (270, 131): 271,
 (271, 260): 272,
 (272, 143): 273,
 (273, 260): 274,
 (274, 132): 275,
 (275, 260): 276}

In [30]:
def encode(text):
    tokens = list(text.encode("utf-8"))
    while len(tokens) >= 2:
        stats = get_stats(tokens)
        #token值越小的代表出现次数越多，最先被合并
        pair = min(stats , key = lambda p: merges.get(p , float("inf")))
        if pair not in merges:
            break
        idx = merges[pair]
        tokens = merge(tokens , pair , idx)
    return tokens


print(encode("h"))

[104]


In [31]:
print(decode(encode("hello world")))

hello world


In [38]:
gpt2pat = re.compile(r"""'s|'t|'re|'ve|'m|'ll|'d| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+""")
print(re.findall(gpt2pat , "hello     world's my dog 123 !!!"))

['hello', '    ', ' world', "'s", ' my', ' dog', ' 123', ' !!!']


In [39]:
example = """
for i in range(1 , 101):
    if i % 3 == 0 and i % 5 == 0:
        print("FizzBuzz")
    elif i % 3 == 0:
        print("Fizz")
    elif i % 5 == 0:
        print("Buzz")
    else:
        print(i)
"""

print(re.findall(gpt2pat , example))

['\n', 'for', ' i', ' in', ' range', '(', '1', ' ,', ' 101', '):', '\n   ', ' if', ' i', ' %', ' 3', ' ==', ' 0', ' and', ' i', ' %', ' 5', ' ==', ' 0', ':', '\n       ', ' print', '("', 'FizzBuzz', '")', '\n   ', ' elif', ' i', ' %', ' 3', ' ==', ' 0', ':', '\n       ', ' print', '("', 'Fizz', '")', '\n   ', ' elif', ' i', ' %', ' 5', ' ==', ' 0', ':', '\n       ', ' print', '("', 'Buzz', '")', '\n   ', ' else', ':', '\n       ', ' print', '(', 'i', ')', '\n']


In [ ]:
# special tokens          <endoftext>